# Testscript.ipynb

A Jupyter notebook for testing the `ArBuWe.py` module.
Doubles as a *"tutorial"* of sorts.

In [ ]:
## Import module

import ArBuWe as abw

In [ ]:
## Initialize input parameters

shapefile_path = "data/FI.shp"
raster_path = "data/gfa_res_curr_density_epsg4326.tif"
weather_start = "2010-06-01"
weather_end = "2010-06-02"

In [ ]:
## Try creating a `Shapefile` and inspect the uniform raster

import matplotlib.pyplot as plt
ec = (0,0,0,1) # Edge color for the vector plots (black)
fc = (0,0,0,0) # Face color for the vector plots (transparent)
lw = 0.2 # Line width for the vector plots

shp = abw.Shapefile(shapefile_path)

f = abw.plot_layout(shp, shp.raster, dpi=100, title="Shapefile")

In [ ]:
## Create weights for testing.

#import random

#weights = {'KU005':0.24, 'KU010':0.73, 'KU009':0.03,}
#weights = {lid:random.uniform(0,1) for lid in shp.data.index}
weights = {lid:1 for lid in shp.data.index}

In [ ]:
## Try creating the cutout

cutout = abw.prepare_cutout(shp, weather_start, weather_end)

In [ ]:
## Try preparing the `layout` raster and inspect nonzero data.

raster, layout = abw.prepare_layout(shp, cutout, weights, raster_path, resampling=5)

In [ ]:
## Plot the raster and layout

fig1 = abw.plot_layout(shp, raster, title="Raster", dpi=300)
fig2 = abw.plot_layout(shp, layout, title="Layout", dpi=300)

In [ ]:
## Process weather data for the cutout.

ambient_temperature, diffuse_irradiation, direct_irradiation = abw.process_weather(cutout, layout)

In [ ]:
# Plot temperature

ambient_temperature.plot()

In [ ]:
# Plot diffuse irradiation

diffuse_irradiation.plot()

In [ ]:
# Plot direct irradiation

import pandas as pd

pd.DataFrame(direct_irradiation).plot()

In [ ]:
## Now do it all again using the convenience function:

ambient_temperature, diffuse_irradiation, direct_irradiation = abw.aggregate_weather(shapefile_path, weights, weather_start, weather_end, raster_path)
pd.DataFrame(ambient_temperature).plot()
pd.DataFrame(diffuse_irradiation).plot()
pd.DataFrame(direct_irradiation).plot()

# ArBuWe development stuff

The below cells were used for developing ArBuWe to be able to handle xarray-level initial heating/cooling demand calculations, as well as checking that the `atlite`
tracking options work for irradiation functions as well.

The below stuff is most likely useless, but I'm leaving it in as potential examples.

In [ ]:
# Raw diffuse irradiation

diffi = cutout.irradiation(
    orientation={"slope": 0.0, "azimuth": 0.0},
    irradiation="diffuse",
    layout=layout, # Remove this line to get xarray level results.
)
diffi.plot()

In [ ]:
# Direct irradiation?

dirs = {
    "horizontal": (0.0, 0.0),
    "north": (90.0, 0.0),
    "east": (90.0, 90.0),
    "south": (90.0, 180.0),
    "west": (90.0, 270.0),
}

diri_none = {
    dir: cutout.irradiation(
        orientation={"slope": sl, "azimuth": az},
        irradiation="direct",
        layout=layout, # Remove this line to get xarray level results.
    )
    for dir, (sl, az) in dirs.items()
}

diri_vertical = {
    dir: cutout.irradiation(
        orientation={"slope": sl, "azimuth": az},
        irradiation="direct",
        tracking="vertical",
        layout=layout, # Remove this line to get xarray level results.
    )
    for dir, (sl, az) in dirs.items()
}

# Plot comparison, "horizontal" shouldn't depend on tracking.

diri_none["horizontal"].plot()
diri_vertical["horizontal"].plot()

In [ ]:
# Plot comparison, no tracking should show 4 different cardinal directions (plus horizontal).

for dir, (sl, az) in dirs.items():
    diri_none[dir].plot()

In [ ]:
# Plot comparison, vertical axis tracking should result in identical radiation for all cardinal directions (plus horizontal).

for dir, (sl, az) in dirs.items():
    diri_vertical[dir].plot()

In [ ]:
# Plot comparison, vertical axis tracking vs no tracking?

for dir, (sl, az) in dirs.items():
    diri_none[dir].plot()
    diri_vertical[dir].plot()

## Seems reasonable enough.

Vertical axis tracking seems to work as intended in `atlite`.
Since we likely won't ever know the actual distribution of window and envelope surface
areas towards the cardinal directions *(or any direction to be frank)* on the
building-stock-scale, we might as well assume the vertical envelope areas to be more or less equally distributed.

Thus, we should be able to use the `vertical` axis tracking option in `atlite` to
calculate the effective solar irradiation on the vertical envelope.
Naturally, this must take into account that not all of the envelope surface faces
the correct way, but that shouldn't be an issue.


### Next steps: Draft the calculations for the heating/cooling demand

What coefficients do we actually need from ArBuMo, and how to efficiently manage
the weather stuff for thermal mass nodes.